# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [5]:
# Your code herei
import sqlite3
import pandas as pd

conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [9]:
# Your code here
cur.execute("""SELECT firstName, lastName 
               FROM employees JOIN offices
               USING (officeCode)
               WHERE offices.city = 'Boston'
               ;""").fetchall()

[('Julie', 'Firrelli'), ('Steve', 'Patterson')]

In [10]:
q = """SELECT firstName, lastName 
               FROM employees JOIN offices
               USING (officeCode)
               WHERE offices.city = 'Boston'
               ;"""
df = pd.read_sql(q, conn)
df

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [14]:
cur.execute("""SELECT offices.officeCode, offices.city
               FROM offices LEFT JOIN employees
               USING (officeCode)
               WHERE employees.officeCode IS NULL
               GROUP BY officeCode
               ;""").fetchall()

[(27, 'Boston')]

In [16]:
# Your code here
q = """SELECT offices.officeCode, offices.city
               FROM offices LEFT JOIN employees
               USING (officeCode)
               WHERE employees.officeCode IS NULL
               GROUP BY officeCode
               ;"""
df = pd.read_sql(q, conn)
df

,officeCode,city
0,27,Boston


In [17]:
alt_q = """SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
               FROM offices AS o 
               LEFT JOIN employees AS e
               USING(officeCode)
               GROUP BY officeCode
               HAVING n_employees = 0;"""
df = pd.read_sql(q, conn)
df

,officeCode,city
0,27,Boston


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold

In [28]:
# Your code here
# Display product line name, product name , in stock number for all LOW STOCK (< 100) products

df = pd.read_sql("""SELECT pl.productLine, p.productName, p.quantityInStock
               FROM productlines AS pl
               JOIN products AS p
               USING(productLine)
               WHERE p.quantityInStock < 100
               ORDER BY p.quantityInStock;""", conn)
df

,productLine,productName,quantityInStock
0,Motorcycles,1960 BSA Gold Star DBD34,15
1,Classic Cars,1968 Ford Mustang,68


In [34]:
# Your code here
# display customer order status
cur.execute("""SELECT  o.status, o.orderNumber, o.orderDate,  c.customerNumber, c.customerName
               FROM customers AS c
               JOIN orders AS o
               USING(customerNumber)
               ORDER BY o.status, o.orderDate
               ;""").fetchall()
df = pd.read_sql("""SELECT  o.status, o.orderNumber, o.orderDate,  c.customerNumber, c.customerName
               FROM customers AS c
               JOIN orders AS o
               USING(customerNumber)
               ORDER BY o.status, o.orderDate
               ;""", conn)
df

,status,orderNumber,orderDate,customerNumber,customerName
0,Cancelled,10167,2003-10-23,448,Scandinavian Gift Ideas
1,Cancelled,10179,2003-11-11,496,Kelly's Gift Shop
2,Cancelled,10248,2004-05-07,131,Land of Toys Inc.
3,Cancelled,10253,2004-06-01,201,"UK Collectables, Ltd."
4,Cancelled,10260,2004-06-16,357,GiftsForHim.com
...,...,...,...,...,...
321,Shipped,10412,2005-05-03,141,Euro+ Shopping Channel
322,Shipped,10413,2005-05-05,175,Gift Depot Inc.
323,Shipped,10416,2005-05-10,386,L'ordine Souveniers
324,Shipped,10418,2005-05-16,412,"Extreme Desk Decorations, Ltd"


In [42]:
# Your code here 
# display our top 10 payments

df = pd.read_sql("""SELECT c.customerName, p.amount
               FROM customers AS c
               JOIN payments AS p
               USING(customerNumber)
               ORDER BY p.amount DESC
               LIMIT 10;""", conn)
df

,customerName,amount
0,Euro+ Shopping Channel,120166.58
1,Euro+ Shopping Channel,116208.40
2,Mini Gifts Distributors Ltd.,111654.40
3,"Dragon Souveniers, Ltd.",105743.00
4,Mini Gifts Distributors Ltd.,101244.59
5,Corporate Gift Ideas Co.,85559.12
6,Mini Gifts Distributors Ltd.,85410.87
7,Herkku Gifts,85024.46
8,Mini Gifts Distributors Ltd.,83598.04
9,"Australian Collectors, Co.",82261.22


## Level Up: Display the names of every individual product that each employee has sold

In [47]:
# Your code here
q = """SELECT employees.lastName, employees.firstName, products.productName 
               FROM employees 
               JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
               JOIN orders USING(customerNumber)
               JOIN orderdetails USING(orderNumber)
               JOIN products using(productCode)
               ;"""

cur.execute(q).fetchall()

df = pd.read_sql(q, conn)
df.shape
df.head()

,lastName,firstName,productName
0,Jennings,Leslie,1958 Setra Bus
1,Jennings,Leslie,1940 Ford Pickup Truck
2,Jennings,Leslie,1939 Cadillac Limousine
3,Jennings,Leslie,1996 Peterbilt 379 Stake Bed with Outrigger
4,Jennings,Leslie,1968 Ford Mustang


## Level Up: Display the Number of Products each employee has sold

In [63]:
# Your code here
q = """SELECT employees.firstName, employees.lastName, SUM(orderdetails.quantityOrdered)
               FROM employees 
               JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
               JOIN orders USING(customerNumber)
               JOIN orderdetails USING(orderNumber)
               GROUP BY employees.employeeNumber
               ORDER BY employees.firstName
               ;"""

cur.execute(q).fetchall()

[('Andy', 'Fixter', 6246),
 ('Barry', 'Jones', 7486),
 ('Foon Yue', 'Tseng', 5016),
 ('George', 'Vanauf', 7423),
 ('Gerard', 'Hernandez', 14231),
 ('Julie', 'Firrelli', 4227),
 ('Larry', 'Bott', 8205),
 ('Leslie', 'Jennings', 11854),
 ('Leslie', 'Thompson', 4056),
 ('Loui', 'Bondur', 6186),
 ('Mami', 'Nishi', 4923),
 ('Martin', 'Gerard', 4180),
 ('Pamela', 'Castillo', 9290),
 ('Peter', 'Marsh', 6632),
 ('Steve', 'Patterson', 5561)]

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!